<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Steps" data-toc-modified-id="Steps-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Steps</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Scrap-pad" data-toc-modified-id="Scrap-pad-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Scrap pad</a></span></li><li><span><a href="#Aggregate-over-month-(not-working)" data-toc-modified-id="Aggregate-over-month-(not-working)-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Aggregate over month (not working)</a></span></li><li><span><a href="#Load-train-&amp;-test-csvs" data-toc-modified-id="Load-train-&amp;-test-csvs-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Load train &amp; test csvs</a></span></li><li><span><a href="#Function-to-add-extra-cols-on-train-&amp;-test-combined-agg-dfs" data-toc-modified-id="Function-to-add-extra-cols-on-train-&amp;-test-combined-agg-dfs-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Function to add extra cols on train &amp; test combined agg dfs</a></span></li><li><span><a href="#Remove-outliers" data-toc-modified-id="Remove-outliers-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Remove outliers</a></span></li><li><span><a href="#Target-encoding-for-outliers" data-toc-modified-id="Target-encoding-for-outliers-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>Target encoding for outliers</a></span></li></ul></li></ul></li></ul></div>

# Steps

1. Import dependencies
2. Import processed df of new transactions
3. Aggregate over card_id 
4. Aggregate over card_id & month_lag - for monthly aggregations of purchases, installments

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

In [13]:
def agg_cat(df):
    cat_agg = {
        'purchase_amount': ['sum', 'max', 'min', 'mean', 'std', 'var'],
        'installments':  ['sum', 'max', 'min', 'mean', 'std', 'var']
    }
    intermdiate_df = (df.groupby('card_id').size().reset_index(name='transactions_count'))
    for c in ['category_2', 'category_3']:
        temp_df1 = df.groupby(c).agg(cat_agg)
        temp_df1.columns = ['_'.join(i).strip() for i in temp_df1.columns.values]
        temp_df1.reset_index(inplace=True)
        pd.merge(intermdiate_df, temp_df1, on='card_id', how='left')
    return intermdiate_df

In [3]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_ptp']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_amax']).dt.days
    return df

In [15]:
for col in ['category_2','category_3']:
    new_temp[col + '_codes'] = new_temp[col].cat.codes
    new_temp[col+'_mean'] = new_temp.groupby([col + '_codes'])['purchase_amount'].transform('mean')

NameError: name 'new_temp' is not defined

In [4]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'purchase_Month', 
                   'subsector_id']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['authorized_flag'] = ['mean', 'sum']
    aggs['purchase_amount'] = ['sum', 'max', 'min', 'mean', 'std', 'var']
    aggs['installments'] = ['sum', 'max', 'min', 'mean', 'std', 'var']
    aggs['month_lag'] = ['sum', 'max', 'min', 'mean', 'std', 'var']
    aggs['purchase_date'] = [np.min, np.max , np.ptp]
    aggs['card_id'] = ['size']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['month_diff'] = ['mean']
    aggs['category_1'] = ['sum', 'mean']
   
 #     Not working due to a bug in pandas 0.23.4
#     cat_agg = {
#         'purchase_amount': ['sum', 'max', 'min', 'mean', 'std', 'var'],
#         'installments':  ['sum', 'max', 'min', 'mean', 'std', 'var']
#     }
#     # aggregate over cat_2, cat_3
#     intermdiate_df = pd.DataFrame
#     for c in ['category_2', 'category_3']:
#         temp_df = df.groupby(c).agg(cat_agg)
#         temp_df.columns = ['_'.join(i).strip() for i in temp_df.columns.values]
#         temp_df.reset_index(inplace=True)
#         pd.merge(intermdiate_df, temp_df, on='card_id', how='left')
        
    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days    
    return new_df

In [5]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    return df

In [6]:
def mark_categorical_cols_for_transactions(df):
    categorical_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1',
       'merchant_category_id', 'merchant_id',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month']
    for c in categorical_cols:
        df[c] = df[c].astype('category').cat.as_ordered()
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

In [20]:
new_merchant_transactions_df = feather.read_dataframe('new_merchant_transactions_df')

ValueError: Categorical categories must be unique

In [9]:
new_merchant_transactions_df = add_extra_cols(new_merchant_transactions_df)

In [10]:
new_merchant_transactions_df = mark_categorical_cols_for_transactions(new_merchant_transactions_df)

In [13]:
%time new_hist_agg = aggregate_by_card_id(new_merchant_transactions_df)

CPU times: user 50.2 s, sys: 978 ms, total: 51.2 s
Wall time: 36.6 s


In [14]:
new_merchant_transactions_df.to_feather('new_merchant_transactions_df')

In [35]:
new_hist_agg["purchase_date_ptp"] = new_hist_agg["purchase_date_ptp"].dt.days    

In [15]:
new_hist_agg.to_feather('new_hist_agg')

In [17]:
new_hist_agg = feather.read_dataframe('new_hist_agg')

In [18]:
new_hist_agg = add_extra_cols_on_agg(new_hist_agg)

### Scrap pad

In [38]:
temp_df = new_merchant_transactions_df[20:30]

In [40]:
temp_df_agg = temp_df.groupby(['card_id','month_lag']).agg({"purchase_amount": ['min', 'max', 'mean', 'std']})

In [41]:
temp_df_agg.columns.values

array([('purchase_amount', 'min'), ('purchase_amount', 'max'), ('purchase_amount', 'mean'),
       ('purchase_amount', 'std')], dtype=object)

In [44]:
['_'.join(c).strip() for c in temp_df_agg.columns.values]

['purchase_amount_min',
 'purchase_amount_max',
 'purchase_amount_mean',
 'purchase_amount_std']

In [58]:
temp_df_agg

purchase_amount                              
                                      min       max      mean       std
card_id         month_lag                                              
C_ID_241a01e9d9 2               -0.461705 -0.294760 -0.378232  0.118048
C_ID_a97720321f 1               -0.202377  0.755743  0.276683  0.677493
C_ID_ef55cf8d4b 1               -0.704834  3.009720  0.683685  1.753241
                2               -0.432253 -0.432253 -0.432253       NaN

In [59]:
temp_df_agg.columns = ['_'.join(c).strip() for c in temp_df_agg.columns.values]

In [61]:
temp_df_agg

purchase_amount_min  purchase_amount_max  \
card_id         month_lag                                             
C_ID_241a01e9d9 2                    -0.461705            -0.294760   
C_ID_a97720321f 1                    -0.202377             0.755743   
C_ID_ef55cf8d4b 1                    -0.704834             3.009720   
                2                    -0.432253            -0.432253   

                           purchase_amount_mean  purchase_amount_std  
card_id         month_lag                                             
C_ID_241a01e9d9 2                     -0.378232             0.118048  
C_ID_a97720321f 1                      0.276683             0.677493  
C_ID_ef55cf8d4b 1                      0.683685             1.753241  
                2                     -0.432253                  NaN

In [64]:
temp_df_agg.groupby('card_id').agg(['mean', 'std'])

purchase_amount_min           purchase_amount_max            \
                               mean       std                mean       std   
card_id                                                                       
C_ID_241a01e9d9           -0.461705       NaN           -0.294760       NaN   
C_ID_a97720321f           -0.202377       NaN            0.755743       NaN   
C_ID_ef55cf8d4b           -0.568543  0.192744            1.288734  2.433842   

                purchase_amount_mean           purchase_amount_std      
                                mean       std                mean std  
card_id                                                                 
C_ID_241a01e9d9            -0.378232       NaN            0.118048 NaN  
C_ID_a97720321f             0.276683       NaN            0.677493 NaN  
C_ID_ef55cf8d4b             0.125716  0.789087            1.753241 NaN

### Aggregate over month (not working)

In [70]:
def aggregate_over_month(df):
    grouped = df.groupby(['card_id','month_lag'])
    agg_func = {
        'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
        'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
        }
    temp_group = grouped.agg(agg_func)
    temp_group.columns = ['_'.join(c).strip() for c in temp_group.columns.values]
    temp_group.reset_index(inplace=True)
    final_group = temp_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ["_".join(c).strip() for c in final_group.columns.values]
    final_group.reset_index(inplace=True)
    return final_group


In [71]:
x = aggregate_over_month(temp_df)

In [73]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

In [74]:
x = aggregate_per_month(temp_df)

In [75]:
x

,card_id,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,...,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std
0,C_ID_00007093c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C_ID_0001238066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C_ID_0001506ef0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C_ID_0001793786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C_ID_000183fdda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,C_ID_0002709b5a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C_ID_000298032a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,C_ID_0002ba3c2e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,C_ID_0002c7c2c1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,C_ID_00032df08f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
def successive_aggregates(df, field1, field2):
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u

In [87]:
y = successive_aggregates(temp_df, 'month_lag', 'purchase_amount')

In [88]:
y

,card_id,month_lag_purchase_amount_mean,month_lag_purchase_amount_min,month_lag_purchase_amount_max,month_lag_purchase_amount_std
0,C_ID_00007093c1,NaN,NaN,NaN,NaN
1,C_ID_0001238066,NaN,NaN,NaN,NaN
2,C_ID_0001506ef0,NaN,NaN,NaN,NaN
3,C_ID_0001793786,NaN,NaN,NaN,NaN
4,C_ID_000183fdda,NaN,NaN,NaN,NaN
5,C_ID_0002709b5a,NaN,NaN,NaN,NaN
6,C_ID_000298032a,NaN,NaN,NaN,NaN
7,C_ID_0002ba3c2e,NaN,NaN,NaN,NaN
8,C_ID_0002c7c2c1,NaN,NaN,NaN,NaN
9,C_ID_00032df08f,NaN,NaN,NaN,NaN


In [82]:
temp_df.reset_index(inplace=True)

In [85]:
temp_df.drop('index', axis=1, inplace=True)

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [86]:
temp_df

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Elapsed,purchased_on_weekend,month_diff
0,1,C_ID_ef55cf8d4b,231,0,1,B,278,M_ID_2ae6547567,1,-0.654885,...,79,False,False,False,False,False,False,1521542253,0,10
1,1,C_ID_ef55cf8d4b,-1,1,3,C,491,M_ID_79692349d6,1,3.009720,...,89,False,False,False,False,False,False,1522440354,0,10
2,1,C_ID_ef55cf8d4b,231,0,1,B,360,M_ID_532b2abc39,2,-0.432253,...,96,False,False,False,False,False,False,1523021302,0,11
3,1,C_ID_ef55cf8d4b,69,0,2,C,87,M_ID_99c89d936e,1,-0.357721,...,83,False,False,False,False,False,False,1521903510,1,10
4,1,C_ID_ef55cf8d4b,-1,1,12,C,690,M_ID_3111c6df35,1,2.126146,...,62,False,False,False,False,False,False,1520077258,1,11
5,1,C_ID_ef55cf8d4b,69,0,1,B,195,M_ID_7fc4685211,1,-0.704834,...,76,False,False,False,False,False,False,1521289951,1,10
6,1,C_ID_241a01e9d9,69,0,3,C,884,M_ID_7c26306ee3,2,-0.461705,...,112,False,False,False,False,False,False,1524424748,1,10
7,1,C_ID_241a01e9d9,188,0,6,C,690,M_ID_7c7d14fd19,2,-0.294760,...,112,False,False,False,False,False,False,1524425651,1,10
8,1,C_ID_a97720321f,69,0,0,A,419,M_ID_7064a8aa68,1,-0.202377,...,61,False,False,False,False,False,False,1519992383,0,11
9,1,C_ID_a97720321f,69,0,0,A,705,M_ID_1f4773aa76,1,0.755743,...,62,False,False,False,False,False,False,1520079298,1,11


### Load train & test csvs

In [9]:
PATH = 'data/elo/'

In [43]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

In [44]:
train.head().T

,0,1,2,3,4
first_active_month,2017-06,2017-01,2016-08,2017-09,2017-11
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749


In [45]:
add_datepart(train, 'first_active_month', drop=False)

In [46]:
train.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
first_active_monthYear,2017,2017,2016,2017,2017
first_active_monthMonth,6,1,8,9,11
first_active_monthWeek,22,52,31,35,44
first_active_monthDay,1,1,1,1,1


In [47]:
add_datepart(test, 'first_active_month', drop=False)

In [10]:
hist_trans = pd.read_csv(f'{PATH}historical_transactions.csv')

In [7]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    cols = ['merchant_id', 'category_3', 'category_2']
    for col in cols:
        df[col] = df[col].fillna(df[col].mode()[0])
    return df

In [11]:
hist_trans = fill_nas_for_transactions_df(hist_trans[:20])

/home/chandrasekhar/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [12]:
hist_trans.head().T

,0,1,2,3,4
authorized_flag,Y,Y,Y,Y,Y
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,N,N,N,N,N
installments,0,0,0,0,0
category_3,A,A,A,A,A
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


In [13]:
add_datepart(hist_trans, 'purchase_date', drop=False)

In [14]:
hist_trans.shape

(20, 27)

In [54]:
new_merchant_transactions_df.shape

(1963031, 29)

In [15]:
hist_trans = add_extra_cols(hist_trans)

In [16]:
hist_trans.shape

(20, 29)

In [17]:
hist_trans = mark_categorical_cols_for_transactions(hist_trans)

In [58]:
hist_trans.to_feather('hist_trans')

In [59]:
hist_trans = feather.read_dataframe('hist_trans')

In [18]:
%time hist_trans_agg = aggregate_by_card_id(hist_trans)

CPU times: user 24.2 ms, sys: 4.01 ms, total: 28.2 ms
Wall time: 27.7 ms


In [19]:
hist_trans.dtypes

authorized_flag                       int64
card_id                            category
city_id                            category
category_1                            int64
installments                          int64
category_3                           object
merchant_category_id               category
merchant_id                        category
month_lag                             int64
purchase_amount                     float64
purchase_date                datetime64[ns]
category_2                          float64
state_id                           category
subsector_id                       category
purchase_Year                      category
purchase_Month                     category
purchase_Week                         int64
purchase_Day                          int64
purchase_Dayofweek                    int64
purchase_Dayofyear                    int64
purchase_Is_month_end                  bool
purchase_Is_month_start                bool
purchase_Is_quarter_end         

In [61]:
hist_trans_agg.shape

(325540, 37)

In [62]:
new_hist_agg.shape

(290001, 39)

In [63]:
hist_trans_agg = add_extra_cols_on_agg(hist_trans_agg)

In [64]:
hist_trans_agg.shape

(325540, 39)

In [65]:
hist_trans_agg.to_feather('hist_trans_agg')

In [66]:
train_df = pd.merge(train, new_hist_agg, how='left', on='card_id')

In [67]:
DataFrameSummary(train_df).summary().T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
first_active_month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201917,75,0,0%,date
card_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201917,201917,0,0%,unique
feature_1,201917,3.10531,1.18616,1,2,3,4,5,201917,5,0,0%,numeric
feature_2,201917,1.74541,0.751362,1,1,2,2,3,201917,3,0,0%,numeric
feature_3,201917,0.565569,0.495683,0,0,1,1,1,201917,2,0,0%,bool
target,201917,-0.393636,3.8505,-33.2193,-0.88311,-0.0234369,0.765453,17.9651,201917,197110,0,0%,numeric
first_active_monthYear,201917,2016.51,0.788199,2011,2016,2017,2017,2018,201917,8,0,0%,numeric
first_active_monthMonth,201917,7.37874,3.34072,1,5,8,10,12,201917,12,0,0%,numeric
first_active_monthWeek,201917,30.9446,13.8454,1,22,31,44,53,201917,21,0,0%,numeric
first_active_monthDay,201917,1,0,1,1,1,1,1,201917,1,0,0%,constant


In [68]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [69]:
train_df2 = join_dfs(train_df, hist_trans_agg, left_on='card_id')

In [174]:
DataFrameSummary(train_df2).summary().T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
first_active_month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201917,75,0,0%,date
card_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201917,201917,0,0%,unique
feature_1,201917,3.10531,1.18616,1,2,3,4,5,201917,5,0,0%,numeric
feature_2,201917,1.74541,0.751362,1,1,2,2,3,201917,3,0,0%,numeric
feature_3,201917,0.565569,0.495683,0,0,1,1,1,201917,2,0,0%,bool
target,201917,-0.393636,3.8505,-33.2193,-0.88311,-0.0234369,0.765453,17.9651,201917,197110,0,0%,numeric
first_active_monthYear,201917,2016.51,0.788199,2011,2016,2017,2017,2018,201917,8,0,0%,numeric
first_active_monthMonth,201917,7.37874,3.34072,1,5,8,10,12,201917,12,0,0%,numeric
first_active_monthWeek,201917,30.9446,13.8454,1,22,31,44,53,201917,21,0,0%,numeric
first_active_monthDay,201917,1,0,1,1,1,1,1,201917,1,0,0%,constant


In [70]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        display(df)

In [179]:
display_all(train_df2.head(1).T)

,0
first_active_month,2017-06-01 00:00:00
card_id,C_ID_92a2005557
feature_1,5
feature_2,2
feature_3,1
target,-0.820283
first_active_monthYear,2017
first_active_monthMonth,6
first_active_monthWeek,22
first_active_monthDay,1


### Function to add extra cols on train & test combined agg dfs

1. Add total installments, purchase_amount, transactions count for the card on all transactions so far
2. Age of the card 
3. difference b/w the card issue and first transaction
4. Difference b/w card issue and last transaction - card active for how many days


In [71]:
test_df = pd.merge(test, new_hist_agg, how='left', on='card_id')

In [72]:
test_df2 = join_dfs(test_df, hist_trans_agg, left_on='card_id')

In [73]:
test_df2.shape

(123623, 94)

In [74]:
train_df2.shape

(201917, 95)

In [183]:
DataFrameSummary(test_df2).summary().T

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
first_active_month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123622,75,1,0.00%,date
card_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123623,123623,0,0%,unique
feature_1,123623,3.10926,1.18911,1,2,3,4,5,123623,5,0,0%,numeric
feature_2,123623,1.7418,0.749195,1,1,2,2,3,123623,3,0,0%,numeric
feature_3,123623,0.564377,0.49584,0,0,1,1,1,123623,2,0,0%,bool
first_active_monthYear,123622,2016.51,0.785047,2011,2016,2017,2017,2018,123622,8,1,0.00%,numeric
first_active_monthMonth,123622,7.38177,3.3375,1,5,8,10,12,123622,12,1,0.00%,numeric
first_active_monthWeek,123622,30.9737,13.8209,1,22,31,44,53,123622,21,1,0.00%,numeric
first_active_monthDay,123622,1,0,1,1,1,1,1,123622,1,1,0.00%,constant
first_active_monthDayofweek,123622,3.18874,1.8966,0,2,3,5,6,123622,7,1,0.00%,numeric


In [75]:
train_df2.to_feather('train_df2')
test_df2.to_feather('test_df2')

In [76]:
train_df2.columns.values

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target',
       'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'first_active_monthElapsed',
       'transactions_count', 'city_id_nunique', 'merchant_category_id_nunique', 'merchant_id_nunique',
       'state_id_nunique', 'purchase_Month_nunique', 'subsector_id_nunique', 'authorized_flag_mean',
       'authorized_flag_sum', 'purchase_amount_sum', 'purchase_amount_max', 'purchase_amount_min',
       'purchase_amount_mean', 'purchase_amount_std', 'purchase_amount_var', 'installments_sum',
       'installments_max', 'installments_min', 'installments_mean', '

In [77]:
for df in [train_df2, test_df2]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_amin_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_amin'] - df['first_active_month']).dt.days

In [78]:
train_df2.shape

(201917, 101)

In [79]:
train_df2.to_feather('train_df2')
test_df2.to_feather('test_df2')

### Remove outliers

In [80]:
train_df2['outliers'] = 0

In [81]:
train_df2.loc[train_df2['target'] < -30, 'outliers'] = 1
train_df2['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encoding for outliers

In [82]:
train_df2.groupby(['feature_1'])['outliers'].mean()

feature_1
1    0.008058
2    0.010610
3    0.010479
4    0.010712
5    0.013145
Name: outliers, dtype: float64

In [83]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df2.groupby([f])['outliers'].mean()
    train_df2[f] = train_df2[f].map(order_label)
    test_df2[f] = test_df2[f].map(order_label)

In [84]:
train_df2.to_feather('train_df2')
test_df2.to_feather('test_df2')